In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import sys
import os
from loguru import logger
from dotenv import load_dotenv
from pydantic import BaseModel
from tqdm.notebook import tqdm

import numpy as np # required for the scikit-learn pipeline to work
import pandas as pd
import plotly.express as px
import mlflow

load_dotenv()

sys.path.insert(0, '..')

from src.viz import blueq_colors

# Controller

In [3]:
class Args(BaseModel):
    testing: bool = False
    log_to_mlflow: bool = True
    experiment_name: str = "FSDS RecSys - L5 - Reco Algo"
    run_name: str = '004-baseline-popular'
    notebook_persist_dp: str = None
    random_seed: int = 41

    user_col: str = 'user_id'
    item_col: str = 'parent_asin'
    rating_col: str = 'rating'
    
    top_K: int = 100
    top_k: int = 10

    batch_size: int = 128

    def init(self):
        self.notebook_persist_dp = os.path.abspath(f"data/{self.run_name}")
        
        if not os.environ.get("MLFLOW_TRACKING_URI"):
            logger.warning(
                f"Environment variable MLFLOW_TRACKING_URI is not set. Setting self.log_to_mlflow to false."
            )
            self.log_to_mlflow = False

        if self.log_to_mlflow:
            logger.info(
                f"Setting up MLflow experiment {self.experiment_name} - run {self.run_name}..."
            )
            import mlflow

            mlflow.set_experiment(self.experiment_name)
            mlflow.start_run(run_name=self.run_name)

        return self
    
args = Args().init()

print(args.model_dump_json(indent=2))

2024-09-14 00:37:47.892 | INFO     | __main__:init:28 - Setting up MLflow experiment FSDS RecSys - L5 - Reco Algo - run 004-baseline-popular...


{
  "testing": false,
  "log_to_mlflow": true,
  "experiment_name": "FSDS RecSys - L5 - Reco Algo",
  "run_name": "004-baseline-popular",
  "notebook_persist_dp": "/home/dvquys/frostmourne/reco-algo/notebooks/data/004-baseline-popular",
  "random_seed": 41,
  "user_col": "user_id",
  "item_col": "parent_asin",
  "rating_col": "rating",
  "top_K": 100,
  "top_k": 10,
  "batch_size": 128
}


# Prep data

In [4]:
train_df = pd.read_parquet("../data/train.parquet")
val_df = pd.read_parquet("../data/val.parquet")

In [5]:
user_col = args.user_col
item_col = args.item_col
rating_col = args.rating_col

# Implememt

In [6]:
popular_items_df = (
    train_df
    .groupby(item_col, as_index=False)
    .size()
    .assign(
        rec_ranking=lambda df: df['size'].rank(method='first', ascending=False).astype(int)
    )
    .sort_values(['rec_ranking'], ascending=[True])
    .head(args.top_K)
)

popular_items_df

,parent_asin,size,rec_ranking
678,B01N3ASPNV,230,1
859,B077GG9D5D,137,2
1615,B087NNPYP3,123,3
281,B004RMK5QG,114,4
829,B07624RBWB,105,5
...,...,...,...
1203,B07PTMKYS7,29,96
959,B07D53QSMK,28,97
1110,B07KRWJCQW,28,98
1273,B07SBX48TY,28,99


In [7]:
recommendations_df = (
    val_df[[user_col]].drop_duplicates().assign(tmp=1)
    .pipe(lambda df: pd.merge(
        df,
        popular_items_df.assign(tmp=1),
        on='tmp',
        how='left'
    ))
    .rename(columns={'size': 'score'})
    [[user_col, item_col, 'score', 'rec_ranking']]
)
    
recommendations_df

,user_id,parent_asin,score,rec_ranking
0,AGS4TR4K5DMBRAFNBYSB2I2RCHHQ,B01N3ASPNV,230,1
1,AGS4TR4K5DMBRAFNBYSB2I2RCHHQ,B077GG9D5D,137,2
2,AGS4TR4K5DMBRAFNBYSB2I2RCHHQ,B087NNPYP3,123,3
3,AGS4TR4K5DMBRAFNBYSB2I2RCHHQ,B004RMK5QG,114,4
4,AGS4TR4K5DMBRAFNBYSB2I2RCHHQ,B07624RBWB,105,5
...,...,...,...,...
345995,AGT3DOVVP5JRBD3JNUK2OG23PMAA,B07PTMKYS7,29,96
345996,AGT3DOVVP5JRBD3JNUK2OG23PMAA,B07D53QSMK,28,97
345997,AGT3DOVVP5JRBD3JNUK2OG23PMAA,B07KRWJCQW,28,98
345998,AGT3DOVVP5JRBD3JNUK2OG23PMAA,B07SBX48TY,28,99


# Evaluate

## Ranking metrics

In [8]:
from src.eval import create_label_df, merge_recs_with_target

In [9]:
label_df = create_label_df(val_df)
label_df

,user_id,parent_asin,rating,rating_rank
31871,AEU4444ZVMLQB4ZXKLDCQL33BZPA,B07JH3LSHN,3.0,1.0
13604,AF3LO27R5D3TCPUNE5U6ZQWMRHAA,B09LTYGYY2,5.0,1.0
27506,AGYCROALU32Q553KYALGCZLDPOSQ,B0039QJKZ8,5.0,1.0
29163,AEZ4WKT6DIOZ5ZC2KXIYU4PUXMDA,B08M2K9K67,5.0,1.0
23463,AF2AAA4CWRVF2IYVE7WB6OOIEMFA,B07SJVCKQW,5.0,1.0
...,...,...,...,...
1412,AGVZRX53LPVHDZQC7SEC7JMHOM3Q,B01MG8P418,4.0,10.0
447,AHZGMQN5OMOHJAJ6F2YPOS66RSDQ,B07MZ6PDG9,5.0,10.0
14909,AEYWWBRMNORKNO6RKCNDN5D5ROTA,B07QKVKCT6,5.0,11.0
464,AHZGMQN5OMOHJAJ6F2YPOS66RSDQ,B06ZY6VHDD,4.0,11.0


In [10]:
eval_df = merge_recs_with_target(recommendations_df, label_df, k=args.top_K)
eval_df

,user_id,parent_asin,score,rec_ranking,rating,rating_rank
33,AE22LPCN47WUTHSG67R6SKN4A4MQ,B01N3ASPNV,230.0,1,0,NaN
42,AE22LPCN47WUTHSG67R6SKN4A4MQ,B077GG9D5D,137.0,2,0,NaN
69,AE22LPCN47WUTHSG67R6SKN4A4MQ,B087NNPYP3,123.0,3,0,NaN
2,AE22LPCN47WUTHSG67R6SKN4A4MQ,B004RMK5QG,114.0,4,0,NaN
40,AE22LPCN47WUTHSG67R6SKN4A4MQ,B07624RBWB,105.0,5,0,NaN
...,...,...,...,...,...,...
349710,AHZZ26USAR7T6VXZ7XIMHB7E3XEQ,B07D53QSMK,28.0,97,0,NaN
349712,AHZZ26USAR7T6VXZ7XIMHB7E3XEQ,B07KRWJCQW,28.0,98,0,NaN
349719,AHZZ26USAR7T6VXZ7XIMHB7E3XEQ,B07SBX48TY,28.0,99,0,NaN
349751,AHZZ26USAR7T6VXZ7XIMHB7E3XEQ,B09CG15F86,28.0,100,0,NaN


### Visualize

In [11]:
from evidently.pipeline.column_mapping import ColumnMapping
from evidently.report import Report
from evidently.metrics import PrecisionTopKMetric
from evidently.metrics import RecallTopKMetric
from evidently.metrics import FBetaTopKMetric
from evidently.metrics import NDCGKMetric
from evidently.metrics import PersonalizationMetric
import warnings

warnings.filterwarnings(
    action='ignore',
    category=FutureWarning,
    module=r'evidently.metrics.recsys.precision_recall_k'
)

from src.viz import color_scheme

In [12]:
column_mapping = ColumnMapping(
    recommendations_type='rank',
    target=rating_col,
    prediction='rec_ranking',
    item_id=item_col,
    user_id=user_col
)

report = Report(metrics=[
    NDCGKMetric(k=args.top_k),
    RecallTopKMetric(k=args.top_K),
    PrecisionTopKMetric(k=args.top_k),
    FBetaTopKMetric(k=args.top_k),
    PersonalizationMetric(k=args.top_k),
], options=[color_scheme])

report.run(
    reference_data=None,
    current_data=eval_df,
    column_mapping=column_mapping
)

evidently_report_fp = f"{args.notebook_persist_dp}/evidently_report.html"
os.makedirs(args.notebook_persist_dp, exist_ok=True)
report.save_html(evidently_report_fp)

if args.log_to_mlflow:
    mlflow.log_artifact(evidently_report_fp)
    for metric_result in report.as_dict()['metrics']:
        metric = metric_result['metric']
        if metric == 'PersonalizationMetric':
            metric_value = float(metric_result['result']['current_value'])
            mlflow.log_metric(f"val_{metric}", metric_value)
            continue
        result = metric_result['result']['current'].to_dict()
        for kth, metric_value in result.items():
            mlflow.log_metric(f"val_{metric}_at_k_as_step", metric_value, step=kth)

# Clean up

In [14]:
all_params = [args]

if args.log_to_mlflow:
    for params in all_params:
        params_dict = params.dict()
        params_ = {f"{params.__repr_name__()}.{k}": v for k, v in params_dict.items()}
        mlflow.log_params(params_)

    mlflow.end_run()

2024/09/14 00:37:54 INFO mlflow.tracking._tracking_service.client: 🏃 View run 004-baseline-popular at: http://localhost:5003/#/experiments/1/runs/719804929f96474fb098d767c1a5b233.
2024/09/14 00:37:54 INFO mlflow.tracking._tracking_service.client: 🧪 View experiment at: http://localhost:5003/#/experiments/1.
